In [1]:
import sys
import numpy as np
import os

sys.path.append('../..')

from hydroml.plot import plot_output, plot_output_compare, plot_error_of_time
from hydroml.model import BaryonModel

from torch.utils.data import DataLoader

from torch import nn

import torch

In [2]:
batch_size = 1
epochs = 20
learning_rate = 1e-5

In [3]:
from hydroml.dataset import TrainDatasetRu
import pickle

with open("DICTP.dat", "rb") as pf:
        DATAP = pickle.load(pf)

# Initial eta grid
etaInit = DATAP["INITIAL_eta"]
# Final eta grid
etaFinal = DATAP["FINAL_eta"]

arrayNetProtonInit = DATAP["0-40"]["INITIAL"][:15000] # 3
arrayNetProtonFinal = DATAP["0-40"]["FINAL"][:15000] # 4

dataset = TrainDatasetRu(arrayNetProtonInit, arrayNetProtonFinal, etaInit, etaFinal, 72, 141, 4)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=1,
    shuffle=True,
)

print(len(data_loader))

15000


In [4]:
model = BaryonModel(len(dataset.etaInit), len(dataset.etaFinal))

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
loss_func = nn.MSELoss(reduction='sum')

In [7]:
for epoch in range(epochs):
    for i, data in enumerate(data_loader):
        keys = data[0]
        values = data[1]

        output = model(keys)

        optimizer.zero_grad()

        loss = loss_func(output, values)

        loss.backward()
        optimizer.step()
        

In [9]:
torch.save(model, "../../models/baryon_model_Ru_200gev.pt")